In [1]:
from argparse import ArgumentParser
from dataloader import load_data, load_localization_data
from test_functions import detection_test, localization_test
from models.network import get_networks
from utils.utils import *
from dataloader import *
from pathlib import Path
from torch.autograd import Variable
import pickle
from test_functions import detection_test
from loss_functions import *
import matplotlib.pyplot as plt
from tqdm import tqdm
parser = ArgumentParser()
parser.add_argument('--config', type=str, default='/lustre/home/acct-eedxw/eedxw-user1/njq/Knowledge_Distillation_AD-main/configs/config.yaml', help="training configuration")


_StoreAction(option_strings=['--config'], dest='config', nargs=None, const=None, default='/lustre/home/acct-eedxw/eedxw-user1/njq/Knowledge_Distillation_AD-main/configs/config.yaml', type=<class 'str'>, choices=None, help='training configuration', metavar=None)

In [2]:
def get_config(config):
    with open(config, 'r') as stream:
        return yaml.load(stream)

def train(config):
    direction_loss_only = config["direction_loss_only"]
    normal_class = config["normal_class"]
    learning_rate = float(config['learning_rate'])
    num_epochs = config["num_epochs"]
    lamda = config['lamda']
    continue_train = config['continue_train']
    last_checkpoint = config['last_checkpoint']

    checkpoint_path = "./outputs/{}/{}/checkpoints/".format(config['experiment_name'], config['dataset_name'])
                                                                                                                                            
    # create directory
    Path(checkpoint_path).mkdir(parents=True, exist_ok=True)
    print('have load dataset')
    train_dataloader, test_dataloader = load_data(config)
    if continue_train:
        vgg, model = get_networks(config, load_checkpoint=True)
    else:
        vgg, model = get_networks(config)

    # Criteria And Optimizers
    if direction_loss_only:
        criterion = DirectionOnlyLoss()
    else:
        criterion = MseDirectionLoss(lamda)
    print('model have been download')
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    if continue_train:
        optimizer.load_state_dict(
            torch.load('{}Opt_{}_epoch_{}.pth'.format(checkpoint_path, normal_class, last_checkpoint)))

    losses = []
    roc_aucs = []
    if continue_train:
        with open('{}Auc_{}_epoch_{}.pickle'.format(checkpoint_path, normal_class, last_checkpoint), 'rb') as f:
            roc_aucs = pickle.load(f)
    print('start to train')
    print('the size of the data set',train_dataloader.__len__())
    for epoch in range(num_epochs + 1):
        model.train()
        epoch_loss = 0
        num_img = 0
        for data in tqdm(train_dataloader):
            num_img += 1
            X = data[0]
            if X.shape[1] == 1:
                X = X.repeat(1, 3, 1, 1)
            X = Variable(X).cuda()
            output_pred = model.forward(X)
            output_real = vgg(X)
            # print('calculate loss')
            total_loss = criterion(output_pred, output_real)

            # Add loss to the list
            epoch_loss += total_loss.item()
            losses.append(total_loss.item())
#             print('start optimizer',num_img)
            # Clear the previous gradients
            optimizer.zero_grad()
            # Compute gradients
            total_loss.backward()
            # Adjust weights
            optimizer.step()
#             print('the image train end',num_img)
            with open("num.txt","a") as f:
                f.write(str(num_img))
        with open("loss.txt","a") as f:
            f.write('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, epoch_loss))
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, epoch_loss))
        # if epoch % 10 == 0:
        roc_auc = detection_test(model, vgg, test_dataloader, config)
        roc_aucs.append(roc_auc)
        print("RocAUC at epoch {}:".format(epoch), roc_auc)
        with open("loss.txt","a") as f:
            f.write("RocAUC at epoch {}_{}:".format(epoch,roc_auc))
        # if epoch % 50 == 0:
        torch.save(model.state_dict(),
                    '{}Cloner_{}_epoch_{}.pth'.format(checkpoint_path, normal_class, epoch))
        print('Successfully save Cloner model to  :{}Opt_{}_epoch_{}.pth'.format(checkpoint_path, normal_class, epoch))
        torch.save(optimizer.state_dict(),
                    '{}Opt_{}_epoch_{}.pth'.format(checkpoint_path, normal_class, epoch))
        print('Successfully save Opt model to   Successfully save mode to  : {}Opt_{}_epoch_{}.pth'.format(checkpoint_path, normal_class, epoch))
        with open('{}Auc_{}_epoch_{}.pickle'.format(checkpoint_path, normal_class, epoch),
                    'wb') as f:
            pickle.dump(roc_aucs, f)

In [ ]:
def main():
    args = parser.parse_args(args=[])
    print('after parser')
    config = get_config(args.config)
    print('get_config')
    train(config)


if __name__ == '__main__':
    print('main')
    main()


main
after parser
get_config
have load dataset


/tmp/ipykernel_131557/1219602501.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


layer : 0 Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer : 1 BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layer : 2 ReLU(inplace=True)
layer : 3 Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer : 4 ReLU(inplace=True)
layer : 5 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
layer : 6 Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer : 7 BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layer : 8 ReLU(inplace=True)
layer : 9 Conv2d(16, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer : 10 ReLU(inplace=True)
layer : 11 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
layer : 12 Conv2d(128, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
layer : 13 BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

100%|██████████| 412/412 [11:52<00:00,  1.73s/it]


epoch [1/201], loss:291.5003
RocAUC at epoch 0: 0.969
Successfully save Cloner model to  :./outputs/new_task/retina/checkpoints/Opt_0_epoch_0.pth
Successfully save Opt model to   Successfully save mode to  : ./outputs/new_task/retina/checkpoints/Opt_0_epoch_0.pth


...

100%|██████████| 412/412 [07:37<00:00,  1.11s/it]


epoch [18/201], loss:115.5299
RocAUC at epoch 17: 0.9701
Successfully save Cloner model to  :./outputs/new_task/retina/checkpoints/Opt_0_epoch_17.pth
Successfully save Opt model to   Successfully save mode to  : ./outputs/new_task/retina/checkpoints/Opt_0_epoch_17.pth


100%|██████████| 412/412 [07:10<00:00,  1.04s/it]


epoch [19/201], loss:114.7869
RocAUC at epoch 18: 0.9714
Successfully save Cloner model to  :./outputs/new_task/retina/checkpoints/Opt_0_epoch_18.pth
Successfully save Opt model to   Successfully save mode to  : ./outputs/new_task/retina/checkpoints/Opt_0_epoch_18.pth


 38%|███▊      | 157/412 [00:55<01:25,  2.97it/s]